In [ ]:
# Transformers installation
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()


In [ ]:
import datasets
from datasets import Dataset
from datasets import load_dataset
import pandas as pd

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Datasets

https://huggingface.co/datasets/mrqa

### Train Size
SQuAD    -> 86,588

NewsQA   -> 74,160

TriviaQA -> 61,688 👈

### Test Size (Taken from validation dataset)
SQuAD    -> 10,507

NewsQA   -> 4,212 👈

TriviaQA -> 7,785



In [ ]:
path = "/Datasets/mrqa-dataset"

mrqa = datasets.load_from_disk(path)
mrqa

DatasetDict({
    train: Dataset({
        features: ['subset', 'context', 'context_tokens', 'qid', 'question', 'question_tokens', 'detected_answers', 'answers'],
        num_rows: 516819
    })
    test: Dataset({
        features: ['subset', 'context', 'context_tokens', 'qid', 'question', 'question_tokens', 'detected_answers', 'answers'],
        num_rows: 9633
    })
    validation: Dataset({
        features: ['subset', 'context', 'context_tokens', 'qid', 'question', 'question_tokens', 'detected_answers', 'answers'],
        num_rows: 58221
    })
})

In [ ]:
df_squad_train = pd.DataFrame(mrqa['train'][:61688])
df_news_train = pd.DataFrame(mrqa['train'][86588:148276])
df_trivia_train = pd.DataFrame(mrqa['train'][160748:222436])

In [ ]:
df_squad_test = pd.DataFrame(mrqa['validation'][:4212])
df_news_test = pd.DataFrame(mrqa['validation'][10507:14719])
df_trivia_test = pd.DataFrame(mrqa['validation'][14719:18931])

In [ ]:
def reformat_answers(df):
    l = []
    for i in range(df.shape[0]):
        l.append({'answer_start': df['detected_answers'][i]['char_spans'][0]['start'],
                'text': df['detected_answers'][i]['text']})

    df['answers_new'] = l
    return df

In [ ]:
df_squad_train = reformat_answers(df_squad_train)
df_news_train = reformat_answers(df_news_train)
df_trivia_train = reformat_answers(df_trivia_train)

df_squad_test = reformat_answers(df_squad_test)
df_news_test = reformat_answers(df_news_test)
df_trivia_test = reformat_answers(df_trivia_test)

In [ ]:
# convert to Dataset type

squad = datasets.DatasetDict(
    {"train": Dataset.from_pandas(df_squad_train),
    "test": Dataset.from_pandas(df_squad_test)
    })

news = datasets.DatasetDict(
    {"train": Dataset.from_pandas(df_news_train),
    "test": Dataset.from_pandas(df_news_test)
    })

trivia = datasets.DatasetDict(
    {"train": Dataset.from_pandas(df_trivia_train),
    "test": Dataset.from_pandas(df_trivia_test)
    })

## Preprocessing

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [ ]:
import collections

n_best = 20
max_answer_length = 30

In [ ]:
max_length = 384
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers_new"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
def preprocess_test_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["qid"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
!pip install evaluate
import evaluate

metric = evaluate.load("squad")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tqdm.auto import tqdm
import numpy as np


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["qid"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["qid"], "answers": ex["answers_new"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

##Squad

In [ ]:
squad

DatasetDict({
    train: Dataset({
        features: ['subset', 'context', 'context_tokens', 'qid', 'question', 'question_tokens', 'detected_answers', 'answers', 'answers_new'],
        num_rows: 61688
    })
    test: Dataset({
        features: ['subset', 'context', 'context_tokens', 'qid', 'question', 'question_tokens', 'detected_answers', 'answers', 'answers_new'],
        num_rows: 4212
    })
})

In [ ]:
train_dataset_squad = squad["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=squad["train"].column_names,
)
len(squad["train"]), len(train_dataset_squad)

  0%|          | 0/62 [00:00<?, ?ba/s]

(61688, 62309)

In [ ]:
test_dataset_squad = squad["test"].map(
    preprocess_test_examples,
    batched=True,
    remove_columns=squad["test"].column_names,
)
len(squad["test"]), len(test_dataset_squad)

  0%|          | 0/5 [00:00<?, ?ba/s]

(4212, 4258)

In [ ]:
tf_train_dataset_squad = train_dataset_squad.to_tf_dataset(
    columns=[
        "input_ids",
        "start_positions",
        "end_positions",
        "attention_mask",
        "token_type_ids",
    ],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_test_dataset_squad = test_dataset_squad.to_tf_dataset(
    columns=["input_ids", "attention_mask", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

Model

In [ ]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 2
num_train_steps = len(tf_train_dataset_squad) * num_train_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.
Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla P100-PCIE-16GB, compute capability 6.0
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [ ]:
from transformers.keras_callbacks import PushToHubCallback

# callback = PushToHubCallback(output_dir="bert-finetuned-squad", tokenizer=tokenizer)

# We're going to do validation afterwards, so no validation mid-training
# model.fit(tf_train_dataset_squad, callbacks=[callback], epochs=num_train_epochs)

model.fit(tf_train_dataset_squad, epochs=num_train_epochs)

Epoch 1/2
3895/3895 [==============================] - 3221s 822ms/step - loss: 1.3641
Epoch 2/2
3895/3895 [==============================] - 3200s 821ms/step - loss: 0.8267


In [ ]:
predictions = model.predict(tf_test_dataset_squad)
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_dataset_squad,
    squad["test"],
)

  0%|          | 0/4212 [00:00<?, ?it/s]

{'exact_match': 78.39506172839506, 'f1': 86.1976255900417}

In [ ]:
predictions = model.predict(tf_test_dataset_news)
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_dataset_news,
    news["test"],
)

  0%|          | 0/4212 [00:00<?, ?it/s]

{'exact_match': 35.5650522317189, 'f1': 51.338634712260216}

In [ ]:
predictions = model.predict(tf_test_dataset_trivia)
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_dataset_trivia,
    trivia["test"],
)

  0%|          | 0/4212 [00:00<?, ?it/s]

{'exact_match': 47.00854700854701, 'f1': 55.93357376749068}

Load from the saved model

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer

model_checkpoint = "huggingface-course/bert-finetuned-squad"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# from transformers import TFAutoModelForQuestionAnswering

saved_model = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
predictions = saved_model.predict(tf_test_dataset_squad)
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_dataset_squad,
    squad["test"],
)

## NewsQA

In [ ]:
news

DatasetDict({
    train: Dataset({
        features: ['subset', 'context', 'context_tokens', 'qid', 'question', 'question_tokens', 'detected_answers', 'answers', 'answers_new'],
        num_rows: 61688
    })
    test: Dataset({
        features: ['subset', 'context', 'context_tokens', 'qid', 'question', 'question_tokens', 'detected_answers', 'answers', 'answers_new'],
        num_rows: 4212
    })
})

In [ ]:
train_dataset_news = news["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=news["train"].column_names,
)

test_dataset_news = news["test"].map(
    preprocess_test_examples,
    batched=True,
    remove_columns=news["test"].column_names,
)

  0%|          | 0/62 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
tf_train_dataset_news = train_dataset_news.to_tf_dataset(
    columns=[
        "input_ids",
        "start_positions",
        "end_positions",
        "attention_mask",
        "token_type_ids",
    ],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)
tf_test_dataset_news = test_dataset_news.to_tf_dataset(
    columns=["input_ids", "attention_mask", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

Model

In [ ]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 2
num_train_steps = len(tf_train_dataset_news) * num_train_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
from transformers.keras_callbacks import PushToHubCallback

# callback = PushToHubCallback(output_dir="bert-finetuned-news", tokenizer=tokenizer)

# We're going to do validation afterwards, so no validation mid-training
# model.fit(tf_train_dataset_news, callbacks=[callback], epochs=num_train_epochs)

model.fit(tf_train_dataset_news, epochs=num_train_epochs)

Epoch 1/2
9724/9724 [==============================] - 13631s 1s/step - loss: 1.1594
Epoch 2/2
9724/9724 [==============================] - 13606s 1s/step - loss: 0.7372


In [ ]:
predictions = model.predict(tf_test_dataset_news)
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_dataset_news,
    news["test"],
)

661/661 [==============================] - 311s 466ms/step


  0%|          | 0/4212 [00:00<?, ?it/s]

{'exact_match': 51.329534662868, 'f1': 66.31889710843322}

In [37]:
predictions = model.predict(tf_test_dataset_squad)
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_dataset_squad,
    squad["test"],
)

267/267 [==============================] - 124s 462ms/step


  0%|          | 0/4212 [00:00<?, ?it/s]

{'exact_match': 64.95726495726495, 'f1': 77.29980348223442}

In [ ]:
predictions = model.predict(tf_test_dataset_trivia)
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_dataset_trivia,
    trivia["test"],
)

1055/1055 [==============================] - 492s 467ms/step


  0%|          | 0/4212 [00:00<?, ?it/s]

{'exact_match': 47.578347578347575, 'f1': 58.41668761553455}

## Trivia

In [ ]:
trivia

DatasetDict({
    train: Dataset({
        features: ['subset', 'context', 'context_tokens', 'qid', 'question', 'question_tokens', 'detected_answers', 'answers', 'answers_new'],
        num_rows: 61688
    })
    test: Dataset({
        features: ['subset', 'context', 'context_tokens', 'qid', 'question', 'question_tokens', 'detected_answers', 'answers', 'answers_new'],
        num_rows: 4212
    })
})

In [ ]:
train_dataset_trivia = trivia["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=trivia["train"].column_names,
)

test_dataset_trivia = trivia["test"].map(
    preprocess_test_examples,
    batched=True,
    remove_columns=trivia["test"].column_names,
)

  0%|          | 0/62 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
tf_train_dataset_trivia = train_dataset_trivia.to_tf_dataset(
    columns=[
        "input_ids",
        "start_positions",
        "end_positions",
        "attention_mask",
        "token_type_ids",
    ],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)
tf_test_dataset_trivia = test_dataset_trivia.to_tf_dataset(
    columns=["input_ids", "attention_mask", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

Model

In [ ]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 2
num_train_steps = len(tf_train_dataset_trivia) * num_train_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="bert-finetuned-trivia", tokenizer=tokenizer)

# We're going to do validation afterwards, so no validation mid-training
model.fit(tf_train_dataset_trivia, callbacks=[callback], epochs=num_train_epochs)

Cloning https://huggingface.co/rhitabrat/bert-finetuned-trivia into local empty directory.


Download file tf_model.h5:   0%|          | 1.58k/416M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/416M [00:00<?, ?B/s]

Epoch 1/2
15432/15432 [==============================] - 11908s 770ms/step - loss: 1.3161
Epoch 2/2
15432/15432 [==============================] - 11843s 767ms/step - loss: 1.0355


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.34k/416M [00:00<?, ?B/s]

To https://huggingface.co/rhitabrat/bert-finetuned-trivia
   e8640cc..c3e3a82  main -> main

   e8640cc..c3e3a82  main -> main



In [ ]:
predictions = model.predict(tf_test_dataset_trivia)
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_dataset_trivia,
    trivia["test"],
)

  0%|          | 0/4212 [00:00<?, ?it/s]

{'exact_match': 61.20607787274454, 'f1': 66.95352138688547}

In [ ]:
predictions = model.predict(tf_test_dataset_squad)
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_dataset_squad,
    squad["test"],
)

  0%|          | 0/4212 [00:00<?, ?it/s]

{'exact_match': 43.68471035137702, 'f1': 52.59061887003808}

In [ ]:
predictions = model.predict(tf_test_dataset_news)
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_dataset_news,
    news["test"],
)

  0%|          | 0/4212 [00:00<?, ?it/s]

{'exact_match': 21.130104463437796, 'f1': 31.24790580998481}